# msticpy Threat Intel Lookup
This notebook describes the use of the Threat Intelligence lookup class in msticpy.
The class allows lookup of individual or multiple IoCs from one or more TI providers.

TILookup is also extensible - you can subclass TIProvider to implement your own custom lookups. You can also subclass the HTTPProvider or KqlProvider classes, which provide support for querying a REST endpoint or Log Analytics table respectively.

You must have msticpy installed to run this notebook:
```
!pip install --upgrade msticpy
```

To use the Azure Sentinel Threat Intel provider you will also need the "azsentinel" components:
```
!pip install --upgrade msticpy[azsentinel]
```

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#msticpy-Threat-Intel-Lookup" data-toc-modified-id="msticpy-Threat-Intel-Lookup-1">msticpy Threat Intel Lookup</a></span></li><li><span><a href="#TILookup-class" data-toc-modified-id="TILookup-class-2">TILookup class</a></span><ul class="toc-item"><li><span><a href="#Available-Providers" data-toc-modified-id="Available-Providers-2.1">Available Providers</a></span></li><li><span><a href="#Configuration-File" data-toc-modified-id="Configuration-File-2.2">Configuration File</a></span></li><li><span><a href="#Looking-up-IoCs" data-toc-modified-id="Looking-up-IoCs-2.3">Looking up IoCs</a></span><ul class="toc-item"><li><span><a href="#lookup_ioc" data-toc-modified-id="lookup_ioc-2.3.1">lookup_ioc</a></span></li><li><span><a href="#Lookup-an-IoC-from-a-single-provider" data-toc-modified-id="Lookup-an-IoC-from-a-single-provider-2.3.2">Lookup an IoC from a single provider</a></span><ul class="toc-item"><li><span><a href="#Or-convert-result-to-a-DataFrame-and-let-pandas-do-the-display-work..." data-toc-modified-id="Or-convert-result-to-a-DataFrame-and-let-pandas-do-the-display-work...-2.3.2.1">Or convert result to a DataFrame and let pandas do the display work...</a></span></li></ul></li></ul></li><li><span><a href="#Lookup-using-all-primary-providers" data-toc-modified-id="Lookup-using-all-primary-providers-2.4">Lookup using all primary providers</a></span></li><li><span><a href="#Provider-Usage" data-toc-modified-id="Provider-Usage-2.5">Provider Usage</a></span><ul class="toc-item"><li><span><a href="#Use-to-do-a-passive-DNS-lookup" data-toc-modified-id="Use-to-do-a-passive-DNS-lookup-2.5.1">Use to do a passive DNS lookup</a></span></li><li><span><a href="#Use-to-do-a-GeoIP-lookup" data-toc-modified-id="Use-to-do-a-GeoIP-lookup-2.5.2">Use to do a GeoIP lookup</a></span></li></ul></li><li><span><a href="#Inferring-IoC-Type-vs.-Specifying-explicity" data-toc-modified-id="Inferring-IoC-Type-vs.-Specifying-explicity-2.6">Inferring IoC Type vs. Specifying explicity</a></span></li><li><span><a href="#Looking-up-Multiple-IoCs" data-toc-modified-id="Looking-up-Multiple-IoCs-2.7">Looking up Multiple IoCs</a></span><ul class="toc-item"><li><span><a href="#lookup_iocs" data-toc-modified-id="lookup_iocs-2.7.1">lookup_iocs</a></span></li><li><span><a href="#Multiple-IP-Lookup-from-single-provider" data-toc-modified-id="Multiple-IP-Lookup-from-single-provider-2.7.2">Multiple IP Lookup from single provider</a></span></li><li><span><a href="#Multiple-IoCs-using-all-providers" data-toc-modified-id="Multiple-IoCs-using-all-providers-2.7.3">Multiple IoCs using all providers</a></span></li><li><span><a href="#Multiple-Mixed-IoC-Types" data-toc-modified-id="Multiple-Mixed-IoC-Types-2.7.4">Multiple Mixed IoC Types</a></span></li></ul></li><li><span><a href="#Specifying-Time-Ranges" data-toc-modified-id="Specifying-Time-Ranges-2.8">Specifying Time Ranges</a></span></li></ul></li></ul></div>

In [1]:
# Imports
import sys
import warnings

from msticpy.common.utility import check_py_version
MIN_REQ_PYTHON = (3,6)
check_py_version(MIN_REQ_PYTHON)

from msticpy import init_notebook
init_notebook(namespace=globals());


# TILookup class
Input can be a single IoC observable or a pandas DataFrame containing
multiple observables. Processing may require a an API key and
processing performance may be limited to a specific number of
requests per minute for the account type that you have.

In [2]:
# TILookup class
display(Markdown("### Constructor\n"))
print(TILookup.__init__.__doc__)
display(Markdown("### Attributes\n"))
for name in [att for att in dir(TILookup) if not att.startswith("_")]:
    display(Markdown(f"#### _{name}()_"))
    print(getattr(TILookup, name).__doc__)
    print()

### Constructor



        Initialize TILookup instance.

        Parameters
        ----------
        primary_providers : Optional[List[TIProvider]], optional
            Primary TI Providers, by default None
        secondary_providers : Optional[List[TIProvider]], optional
            Secondary TI Providers, by default None
        providers: Optional[List[str]], optional
            List of provider names to load, by default all available
            providers are loaded. To see the list of available providers
            call `TILookup.list_available_providers()`.
            Note: if primary_provides or secondary_providers is specified
            This will override the providers list.

        


### Attributes


#### _add_provider()_


        Add a TI provider to the current collection.

        Parameters
        ----------
        provider : TIProvider
            Provider instance
        name : str, optional
            The name to use for the provider (overrides the class name
            of `provider`)
        primary : bool, optional
            "primary" or "secondary" if False, by default "primary"

        



...

Reload provider settings from config.



#### _reload_providers()_


        Reload providers based on currrent settings in config.

        Parameters
        ----------
        clear_keyring : bool, optional
            Clears any secrets cached in keyring, by default False

        



#### _result_to_df()_


        Return DataFrame representation of IoC Lookup response.

        Parameters
        ----------
        ioc_lookup : Tuple[bool, List[Tuple[str, LookupResult]]]
            Output from `lookup_ioc`

        Returns
        -------
        pd.DataFrame
            The response as a DataFrame with a row for each
            provider response.

        



## Available Providers
The **msticpy** TI Provider library can lookup IoCs in multiple providers.

The list below shows the current set of providers.

In [3]:
TILookup.list_available_providers()

AzSTI
GreyNoise
OPR
OTX
Tor
VirusTotal
XForce


You can view the list of supported query types for each provider with the `show_query_types=True` parameter

In [4]:
TILookup.list_available_providers(show_query_types=True)

AzSTI
Azure Sentinel TI provider class. Supported query types:
	ioc_type=dns
	ioc_type=file_hash
	ioc_type=hostname
	ioc_type=ipv4
	ioc_type=ipv6
	ioc_type=linux_path
	ioc_type=md5_hash
	ioc_type=sha1_hash
	ioc_type=sha256_hash
	ioc_type=url
	ioc_type=windows_path
GreyNoise
GreyNoise Lookup. Supported query types:
	ioc_type=ipv4
	ioc_type=ipv4, ioc_query_type=full
	ioc_type=ipv4, ioc_query_type=quick
OPR
Open PageRank Lookup. Supported query types:
	ioc_type=dns
OTX
AlientVault OTX Lookup. Supported query types:
	ioc_type=dns
	ioc_type=dns, ioc_query_type=geo
	ioc_type=dns, ioc_query_type=passivedns
	ioc_type=file_hash
	ioc_type=hostname
	ioc_type=ipv4
	ioc_type=ipv4, ioc_query_type=geo
	ioc_type=ipv4, ioc_query_type=passivedns
	ioc_type=ipv6
	ioc_type=ipv6, ioc_query_type=geo
	ioc_type=ipv6, ioc_query_type=passivedns
	ioc_type=md5_hash
	ioc_type=sha1_hash
	ioc_type=sha256_hash
	ioc_type=url
Tor
Tor Exit Nodes Lookup. Supported query types:
	ioc_type=ipv4
VirusTotal
VirusTotal Lookup. 

## Loading TIProviders

Calling TILookup with no parameters will load all of the available providers
that have a configuration entry in `msticpyconfig.yaml` (see next section)

In [5]:
# load all configured providers
ti_lookup = TILookup()
ti_lookup.provider_status

# Restricting which providers get loaded
#ti_lookup = TILookup(providers=["VirusTotal", "XForce"])

Attempting to sign-in with environment variable credentials...
Using Open PageRank. See https://www.domcop.com/openpagerank/what-is-openpagerank
Please wait. Loading Kqlmagic extension...


['OTX - AlientVault OTX Lookup. (primary)',
 'VirusTotal - VirusTotal Lookup. (primary)',
 'XForce - IBM XForce Lookup. (primary)',
 'GreyNoise - GreyNoise Lookup. (primary)',
 'AzSTI - Azure Sentinel TI provider class. (primary)',
 'OPR - Open PageRank Lookup. (secondary)']

## Configuration File
You can configure primary and secondary providers. 
Primary providers are used by default.

You may need to supply an authorization (API) key and in some cases a user ID for each provider.

For LogAnalytics/Azure Sentinel providers, you will need the workspace ID and tenant ID and will need to authenticate in order to access the data (although if you have an existing authenticated connection with the same workspace/tenant, this connection will be re-used).

The configuration file is read from the current directory.

Alternatively, you can specify a location for this file in an environment variable `MSTICPYCONFIG`.

If you need to create a config file, uncomment the lines in the following cell.<br>
### <font color="red">Warning - this will overwrite a file of the same name in the current directory</font>

Delete any provider entries that you do not want to use and add the missing parameters for your providers. 

In [13]:
# %%writefile msticpyconfig.yaml
# QueryDefinitions:

# TIProviders:
#   OTX:
#     Args:
#       AuthKey: "your-otx-key"
#     Primary: True
#     Provider: "OTX" # Explicitly name provider to override
#   VirusTotal:
#     Args:
#       AuthKey: "your-vt-key"
#     Primary: True
#     Provider: "VirusTotal"
#   XForce:
#     Args:
#       ApiID: "your-xforce-id"
#       AuthKey: "your-xforce-key"
#     Primary: True
#     Provider: "XForce"
#   GreyNoise:
#     Args:
#       AuthKey: ""
#     Primary: True
#     Provider: "GreyNoise"
#   AzureSentinel:
#     # Note if you do not specify any settings in the Args key for the AzureSentinel
#     # provider, it will default to using your default Azure Sentinel workspace.
#     Args:
#       WorkspaceID: "your-azure-sentinel-workspace-id"
#       TenantID: "your-azure-sentinel-tenant-id"
#     Primary: True
#     Provider: "AzSTI"

Reload providers to pick up new settings

In [6]:
ti_lookup.reload_providers()
ti_lookup.provider_status

Settings reloaded. Use reload_providers to update settings for loaded providers.
Using Open PageRank. See https://www.domcop.com/openpagerank/what-is-openpagerank


['OTX - AlientVault OTX Lookup. (primary)',
 'VirusTotal - VirusTotal Lookup. (primary)',
 'XForce - IBM XForce Lookup. (primary)',
 'GreyNoise - GreyNoise Lookup. (primary)',
 'AzSTI - Azure Sentinel TI provider class. (primary)',
 'OPR - Open PageRank Lookup. (secondary)']

## Looking up IoCs

### lookup_ioc
To lookup a single IoC.
```
ti_lookup.lookup_ioc(
    observable: str = None,
    ioc_type: str = None,
    ioc_query_type: str = None,
    providers: List[str] = None,
    prov_scope: str = 'primary',
    **kwargs,
) -> Tuple[bool, List[Tuple[str, msticpy.sectools.tiproviders.ti_provider_base.LookupResult]]]

Lookup single IoC in active providers.

Parameters
----------
observable : str
    IoC observable
    (`ioc` is also an alias for observable)
ioc_type : str, optional
    One of IoCExtract.IoCType, by default None
    If none, the IoC type will be inferred
ioc_query_type: str, optional
    The ioc query type (e.g. rep, info, malware)
providers: List[str]
    Explicit list of providers to use
prov_scope : str, optional
    Use primary, secondary or all providers, by default "primary"
kwargs :
    Additional arguments passed to the underlying provider(s)

Returns
-------
Tuple[bool, List[Tuple[str, LookupResult]]]
    The result returned as a tuple(bool, list):
    bool indicates whether a TI record was found in any provider
    list has an entry for each provider result
```

In [8]:
# Uncomment this and run to see the document string
# ti_lookup.lookup_ioc?

### Lookup an IoC from a single provider
And show the output

In [7]:
result = ti_lookup.lookup_ioc(observable="52.183.120.194", providers=["AzSTI", "XForce"])
ti_lookup.result_to_df(result)

,Ioc,IocType,QuerySubtype,Provider,Result,Severity,Details,RawResult,Reference,Status
XForce,52.183.120.194,ipv4,None,XForce,True,warning,"{'score': 1, 'cats': {}, 'categoryDescriptions': {}, 'reason': 'Regional Internet Registry', 're...","{'ip': '52.183.120.194', 'history': [{'created': '2012-03-22T07:26:00.000Z', 'reason': 'Regional...",https://api.xforce.ibmcloud.com/ipr/52.183.120.194,0
AzSTI,52.183.120.194,ipv4,None,AzSTI,False,information,Not found.,None,None,0


In [8]:
result = ti_lookup.lookup_ioc(observable="52.183.120.194")
ti_lookup.result_to_df(result).T

,OTX,VirusTotal,XForce,GreyNoise,AzSTI
Ioc,52.183.120.194,52.183.120.194,52.183.120.194,52.183.120.194,52.183.120.194
IocType,ipv4,ipv4,ipv4,ipv4,ipv4
QuerySubtype,None,None,None,None,None
Provider,OTX,VirusTotal,XForce,GreyNoise,AzSTI
Result,True,True,True,False,False
Severity,information,information,warning,information,information
Details,"{'pulse_count': 0, 'sections_available': ['general', 'geo', 'reputation', 'url_list', 'passive_d...","{'verbose_msg': 'IP address in dataset', 'response_code': 1, 'detected_urls': [], 'positives': 0...","{'score': 1, 'cats': {}, 'categoryDescriptions': {}, 'reason': 'Regional Internet Registry', 're...",Not found.,Not found.
RawResult,"{'whois': 'http://whois.domaintools.com/52.183.120.194', 'reputation': 0, 'indicator': '52.183.1...","{'undetected_urls': [['http://52.183.120.194/', 'a81857a8287017ddadd3cf776a1538e136b5bff261db758...","{'ip': '52.183.120.194', 'history': [{'created': '2012-03-22T07:26:00.000Z', 'reason': 'Regional...",<Response [404]>,None
Reference,https://otx.alienvault.com/api/v1/indicators/IPv4/52.183.120.194/general,https://www.virustotal.com/vtapi/v2/ip-address/report,https://api.xforce.ibmcloud.com/ipr/52.183.120.194,https://api.GreyNoise.io/v3/community/52.183.120.194,None
Status,0,0,0,404,0


In [9]:
import pprint
pp = pprint.PrettyPrinter(indent=2)

result, details = ti_lookup.lookup_ioc(observable="38.75.137.9", providers=["OTX"])

# the details is a list (since there could be multiple responses for an IoC)
for provider, detail in details:
    print(provider)
    detail.summary
# Un-comment to view raw response
#     print("\nRaw Results")
#     pp.pprint(detail.raw_result)


OTX
ioc: 38.75.137.9 ( ipv4 )
result: True
{   'names': [   'Underminer.EK - Exploit Kit IOC Feed',
                 '<script '
                 'src="https://otx.alienvault.com/pulse/5d4d8ccdbe24622d01f9ce9f.js"></script>',
                 'Underminer.EK - Exploit Kit IOC Feed',
                 'Underminer EK'],
    'pulse_count': 4,
    'references': [   [],
                      [],
                      [],
                      [   'https://blog.malwarebytes.com/threat-analysis/2019/07/exploit-kits-summer-2019-review/']],
    'tags': [['Underminer.EK'], ['Underminer.EK'], ['Underminer.EK'], []]}
reference:  https://otx.alienvault.com/api/v1/indicators/IPv4/38.75.137.9/general


#### Or convert result to a DataFrame and let pandas do the display work...

In [10]:
result = ti_lookup.lookup_ioc(observable="38.75.137.9", providers=["OTX"])
ti_lookup.result_to_df(result).T

,OTX
Ioc,38.75.137.9
IocType,ipv4
QuerySubtype,None
Provider,OTX
Result,True
Severity,high
Details,"{'pulse_count': 4, 'names': ['Underminer.EK - Exploit Kit IOC Feed', '<script src=""https://otx.a..."
RawResult,"{'whois': 'http://whois.domaintools.com/38.75.137.9', 'reputation': 0, 'indicator': '38.75.137.9..."
Reference,https://otx.alienvault.com/api/v1/indicators/IPv4/38.75.137.9/general
Status,0


In [11]:
# Extract a single field (RawResult) from the dataframe (.iloc[0] is to select the row)
ti_lookup.result_to_df(result)["RawResult"].iloc[0]

{'whois': 'http://whois.domaintools.com/38.75.137.9',
 'reputation': 0,
 'indicator': '38.75.137.9',
 'type': 'IPv4',
 'type_title': 'IPv4',
 'base_indicator': {'id': 2127020821,
  'indicator': '38.75.137.9',
  'type': 'IPv4',
  'title': '',
  'description': '',
  'content': '',
  'access_type': 'public',
  'access_reason': ''},
 'pulse_info': {'count': 4,
  'pulses': [{'id': '5d4d8ccdbe24622d01f9ce9f',
    'name': 'Underminer.EK - Exploit Kit IOC Feed',
    'description': 'IPs and hostnames for the Exploit Kit known as Underminer.EK.',
    'modified': '2021-04-14T19:27:12.332000',
    'created': '2019-08-09T15:10:05.503000',
    'tags': ['Underminer.EK'],
    'references': [],
    'public': 1,
    'adversary': '',
    'targeted_countries': [],
    'malware_families': [],
    'attack_ids': [],
    'industries': [],
    'TLP': 'white',
    'cloned_from': None,
    'export_count': 770,
    'upvotes_count': 0,
    'downvotes_count': 0,
    'votes_count': 0,
    'locked': False,
    'pulse

## Lookup using all primary providers

In [12]:
result = ti_lookup.lookup_ioc(observable="188.127.231.124")
ti_lookup.result_to_df(result)

,Ioc,IocType,QuerySubtype,Provider,Result,Severity,Details,RawResult,Reference,Status
OTX,188.127.231.124,ipv4,None,OTX,True,high,"{'pulse_count': 7, 'names': ['IOC 06/12/2019', '27/11/2019 IOCs', 'Ransomware IP Blocklist', 'Lo...","{'whois': 'http://whois.domaintools.com/188.127.231.124', 'reputation': 0, 'indicator': '188.127...",https://otx.alienvault.com/api/v1/indicators/IPv4/188.127.231.124/general,0
VirusTotal,188.127.231.124,ipv4,None,VirusTotal,True,information,"{'verbose_msg': 'IP address in dataset', 'response_code': 1, 'detected_urls': ['http://188.127.2...","{'asn': 56694, 'undetected_urls': [['https://radiosfera.net/subject/5391/mikroshema-la4280', '90...",https://www.virustotal.com/vtapi/v2/ip-address/report,0
XForce,188.127.231.124,ipv4,None,XForce,True,warning,"{'score': 1, 'cats': {}, 'categoryDescriptions': {}, 'reason': 'Regional Internet Registry', 're...","{'ip': '188.127.231.124', 'history': [{'created': '2012-03-22T07:26:00.000Z', 'reason': 'Regiona...",https://api.xforce.ibmcloud.com/ipr/188.127.231.124,0
GreyNoise,188.127.231.124,ipv4,None,GreyNoise,False,information,Not found.,<Response [404]>,https://api.GreyNoise.io/v3/community/188.127.231.124,404
AzSTI,188.127.231.124,ipv4,None,AzSTI,False,information,Not found.,None,None,0


## Provider Usage
This shows the supported IoC Types.

In some cases an IoC type will also support special types of sub-query such as geo-ip and passive-dns

In [13]:
display(ti_lookup.provider_status)
ti_lookup.loaded_providers["AzSTI"].usage()

['OTX - AlientVault OTX Lookup. (primary)',
 'VirusTotal - VirusTotal Lookup. (primary)',
 'XForce - IBM XForce Lookup. (primary)',
 'GreyNoise - GreyNoise Lookup. (primary)',
 'AzSTI - Azure Sentinel TI provider class. (primary)',
 'OPR - Open PageRank Lookup. (secondary)']

Azure Sentinel TI provider class. Supported query types:
	ioc_type=dns
	ioc_type=file_hash
	ioc_type=hostname
	ioc_type=ipv4
	ioc_type=ipv6
	ioc_type=linux_path
	ioc_type=md5_hash
	ioc_type=sha1_hash
	ioc_type=sha256_hash
	ioc_type=url
	ioc_type=windows_path


In [14]:
ti_lookup.provider_usage()

Primary providers
-----------------

Provider class: OTX
AlientVault OTX Lookup. Supported query types:
	ioc_type=dns
	ioc_type=dns, ioc_query_type=geo
	ioc_type=dns, ioc_query_type=passivedns
	ioc_type=file_hash
	ioc_type=hostname
	ioc_type=ipv4
	ioc_type=ipv4, ioc_query_type=geo
	ioc_type=ipv4, ioc_query_type=passivedns
	ioc_type=ipv6
	ioc_type=ipv6, ioc_query_type=geo
	ioc_type=ipv6, ioc_query_type=passivedns
	ioc_type=md5_hash
	ioc_type=sha1_hash
	ioc_type=sha256_hash
	ioc_type=url

Provider class: VirusTotal
VirusTotal Lookup. Supported query types:
	ioc_type=dns
	ioc_type=file_hash
	ioc_type=ipv4
	ioc_type=md5_hash
	ioc_type=sha1_hash
	ioc_type=sha256_hash
	ioc_type=url

Provider class: XForce
IBM XForce Lookup. Supported query types:
	ioc_type=dns
	ioc_type=dns, ioc_query_type=malware
	ioc_type=dns, ioc_query_type=passivedns
	ioc_type=dns, ioc_query_type=whois
	ioc_type=file_hash
	ioc_type=hostname, ioc_query_type=whois
	ioc_type=ipv4
	ioc_type=ipv4, ioc_query_type=malware
	ioc_

### Use to do a passive DNS lookup

In [15]:
result = ti_lookup.lookup_ioc(observable="38.75.137.9", ico_type="ipv4", ioc_query_type="passivedns", providers=["XForce"])
print(result)
print("\nProvider result:")
result[1][0][1].raw_result

(True, [('XForce', LookupResult(ioc='38.75.137.9', ioc_type='ipv4', safe_ioc='38.75.137.9', query_subtype='passivedns', provider='XForce', result=True, severity='information', details={'records': 1}, raw_result={'Passive': {'query': '0x00000000000000000000ffff264b8909', 'records': []}, 'RDNS': ['9-137-75-38.clients.gthost.com'], 'total_rows': 1}, reference='https://api.xforce.ibmcloud.com/resolve/38.75.137.9', status=0))])

Provider result:


{'Passive': {'query': '0x00000000000000000000ffff264b8909', 'records': []},
 'RDNS': ['9-137-75-38.clients.gthost.com'],
 'total_rows': 1}

### Use to do a GeoIP lookup

In [16]:
result = ti_lookup.lookup_ioc(observable="38.75.137.9", ico_type="ipv4", ioc_query_type="geo", providers=["OTX"])
print(result)
print("\nProvider result:")
result[1][0][1].raw_result

(True, [('OTX', LookupResult(ioc='38.75.137.9', ioc_type='ipv4', safe_ioc='38.75.137.9', query_subtype='geo', provider='OTX', result=True, severity='information', details={}, raw_result={'asn': 'AS63023 AS-GLOBALTELEHOST', 'city_data': True, 'city': 'Los Angeles', 'region': 'CA', 'continent_code': 'NA', 'country_code3': 'USA', 'country_code2': 'US', 'subdivision': 'CA', 'latitude': 34.0544, 'postal_code': '90009', 'longitude': -118.244, 'accuracy_radius': 1000, 'country_code': 'US', 'country_name': 'United States of America', 'dma_code': 803, 'charset': 0, 'area_code': 0, 'flag_url': '/assets/images/flags/us.png', 'flag_title': 'United States of America'}, reference='https://otx.alienvault.com/api/v1/indicators/IPv4/38.75.137.9/geo', status=0))])

Provider result:


{'asn': 'AS63023 AS-GLOBALTELEHOST',
 'city_data': True,
 'city': 'Los Angeles',
 'region': 'CA',
 'continent_code': 'NA',
 'country_code3': 'USA',
 'country_code2': 'US',
 'subdivision': 'CA',
 'latitude': 34.0544,
 'postal_code': '90009',
 'longitude': -118.244,
 'accuracy_radius': 1000,
 'country_code': 'US',
 'country_name': 'United States of America',
 'dma_code': 803,
 'charset': 0,
 'area_code': 0,
 'flag_url': '/assets/images/flags/us.png',
 'flag_title': 'United States of America'}

## Inferring IoC Type vs. Specifying explicity
If you do a lookup without specifying a type, TILookup will try to infer the type by matching regexes. There are patterns for all supported types but there are some caveats:

- The match is not 100% foolproof - e.g. some URLs and hash types may be misidentified.
- The inference adds an overhead to each lookup.

If you know the type that you want to look up, it is always better to explicitly include it.
- For single IoC lookup, use the `ioc_type` parameter.
- For multiple IoC lookups (see below), supply either:
  - a DataFrame with a column that specifies the type for each entry
  - a dictionary of the form `{ioc_observable: ioc_type}`

## Looking up Multiple IoCs

### lookup_iocs
```
Signature:
ti_lookup.lookup_iocs(
    data: Union[pandas.core.frame.DataFrame, Mapping[str, str], Iterable[str]],
    obs_col: str = None,
    ioc_type_col: str = None,
    ioc_query_type: str = None,
    providers: List[str] = None,
    prov_scope: str = 'primary',
    **kwargs,
) -> pandas.core.frame.DataFrame

Lookup a collection of IoCs.

Parameters
----------
data : Union[pd.DataFrame, Mapping[str, str], Iterable[str]]
    Data input in one of three formats:
    1. Pandas dataframe (you must supply the column name in
    `obs_col` parameter)
    2. Mapping (e.g. a dict) of [observable, IoCType]
    3. Iterable of observables - IoCTypes will be inferred
obs_col : str, optional
    DataFrame column to use for observables, by default None
ioc_type_col : str, optional
    DataFrame column to use for IoCTypes, by default None
ioc_query_type: str, optional
    The ioc query type (e.g. rep, info, malware)
providers: List[str]
    Explicit list of providers to use
prov_scope : str, optional
    Use primary, secondary or all providers, by default "primary"
kwargs :
    Additional arguments passed to the underlying provider(s)

Returns
-------
pd.DataFrame
    DataFrame of results
```

In [19]:
# Uncomment this and run to see the document string
# ti_lookup.lookup_iocs?

### Multiple IP Lookup from single provider

In [27]:

ioc_ips = [
    "51.75.29.61",
    "190.2.144.45"
    "52.183.120.194",
    "146.56.231.70",
    "1.2.3.4",
    "109.236.89.61",
    "1.2.3.5",
]

ti_lookup.lookup_iocs(data=ioc_ips, providers="AzSTI")

,Ioc,IocType,QuerySubtype,Reference,Result,Status,Severity,Details,RawResult,Provider
0,1.2.3.4,ipv4,None,ThreatIntelligenceIndicator | where TimeGenerated >= datetime(2021-03-22T18:24:18.972814Z) | w...,False,0.0,0.0,Not found.,NaN,AzSTI
1,51.75.29.61,ipv4,None,ThreatIntelligenceIndicator | where TimeGenerated >= datetime(2021-03-22T18:24:18.972814Z) | w...,False,0.0,0.0,Not found.,NaN,AzSTI
2,1.2.3.5,ipv4,None,ThreatIntelligenceIndicator | where TimeGenerated >= datetime(2021-03-22T18:24:18.972814Z) | w...,False,0.0,0.0,Not found.,NaN,AzSTI
3,109.236.89.61,ipv4,None,ThreatIntelligenceIndicator | where TimeGenerated >= datetime(2021-03-22T18:24:18.972814Z) | w...,True,0.0,2.0,"{'Action': 'alert', 'ThreatType': 'Botnet', 'ThreatSeverity': nan, 'Active': True, 'Description'...","{'IndicatorId': '2E9C9B5884CE9D1AAECA21C5EB772C30C3C84093C452AA038868AD4EEF50E83E', 'TimeGenerat...",AzSTI
4,109.236.89.61,ipv4,None,ThreatIntelligenceIndicator | where TimeGenerated >= datetime(2021-03-22T18:24:18.972814Z) | w...,True,0.0,2.0,"{'Action': 'alert', 'ThreatType': 'Botnet', 'ThreatSeverity': nan, 'Active': True, 'Description'...","{'IndicatorId': 'AA38B8D61F8587B779AABF6C4F2DBD226C7704600A5CBBB7779A7D0E7AFA05C4', 'TimeGenerat...",AzSTI
5,109.236.89.61,ipv4,None,ThreatIntelligenceIndicator | where TimeGenerated >= datetime(2021-03-22T18:24:18.972814Z) | w...,True,0.0,2.0,"{'Action': 'alert', 'ThreatType': 'Botnet', 'ThreatSeverity': nan, 'Active': True, 'Description'...","{'IndicatorId': 'E9DD356FF6855D16952067E4A35F0AAA1A8FF4E955E20B249131BD0EF7115A1E', 'TimeGenerat...",AzSTI
6,109.236.89.61,ipv4,None,ThreatIntelligenceIndicator | where TimeGenerated >= datetime(2021-03-22T18:24:18.972814Z) | w...,True,0.0,2.0,"{'Action': 'alert', 'ThreatType': 'Botnet', 'ThreatSeverity': nan, 'Active': True, 'Description'...","{'IndicatorId': '0612D0A787F9B2EA5FC1206C3727B3ADAD45D8AEDE2D90ADEC1C3B1590A609E0', 'TimeGenerat...",AzSTI
7,109.236.89.61,ipv4,None,ThreatIntelligenceIndicator | where TimeGenerated >= datetime(2021-03-22T18:24:18.972814Z) | w...,True,0.0,2.0,"{'Action': 'alert', 'ThreatType': 'Botnet', 'ThreatSeverity': nan, 'Active': True, 'Description'...","{'IndicatorId': '4DDD4BC9AD3844756B4DF7FAB23E9E9B917124D0A257BB405EF5715DEAA3D61B', 'TimeGenerat...",AzSTI
8,109.236.89.61,ipv4,None,ThreatIntelligenceIndicator | where TimeGenerated >= datetime(2021-03-22T18:24:18.972814Z) | w...,True,0.0,2.0,"{'Action': 'alert', 'ThreatType': 'Botnet', 'ThreatSeverity': nan, 'Active': True, 'Description'...","{'IndicatorId': 'CAFF80F6C8FD64CDA73AE68E382C6A7B6C94CA257C897324E6DB78D8C2106588', 'TimeGenerat...",AzSTI
9,109.236.89.61,ipv4,None,ThreatIntelligenceIndicator | where TimeGenerated >= datetime(2021-03-22T18:24:18.972814Z) | w...,True,0.0,2.0,"{'Action': 'alert', 'ThreatType': 'Botnet', 'ThreatSeverity': nan, 'Active': True, 'Description'...","{'IndicatorId': 'C72FAFC4A20054F73B343418F5EBE887610D257FFD51439392CC0376AB3B7173', 'TimeGenerat...",AzSTI


### Multiple IoCs using all providers
Output sorted by IoC

Note that these URLs were picked randomly from the TI databases of the three providers used. In most cases the IoC is found by only that provider, which 

In [19]:
ioc_urls = [
    "http://cheapshirts.us/zVnMrG.php",
    "http://chinasymbolic.com/i9jnrc",
    "https://hotel-bristol.lu/dlry/MAnJIPnY/",
    "http://businesstobuy.net",
    "http://append.pl/srh9xsz",
    "http://104.248.196.145/apache2",
    "http://ajaraheritage.ge/g7cberv",
    "http://cic-integration.com/hjy93JNBasdas",
    "https://google.com",  # benign
    "https://microsoft.com",  # benign
    "https://python.org",  # benign
]
results = ti_lookup.lookup_iocs(data=ioc_urls)
results.sort_values("Ioc")

,Ioc,IocType,SafeIoc,QuerySubtype,Provider,Result,Severity,Details,RawResult,Reference,Status
9,http://104.248.196.145/apache2,url,NaN,None,AzSTI,False,0,Not found.,NaN,ThreatIntelligenceIndicator | where TimeGenerated >= datetime(2021-03-22T18:22:19.058146Z) | w...,0
5,http://104.248.196.145/apache2,url,http://104.248.196.145/apache2,None,VirusTotal,False,information,No Content,<Response [204]>,https://www.virustotal.com/vtapi/v2/url/report,204
5,http://104.248.196.145/apache2,url,http://104.248.196.145/apache2,None,XForce,False,information,Not found.,<Response [404]>,https://api.xforce.ibmcloud.com/url/http://104.248.196.145/apache2,404
5,http://104.248.196.145/apache2,url,http%3A%2F%2F104.248.196.145%2Fapache2,None,OTX,True,information,"{'pulse_count': 0, 'sections_available': ['general', 'url_list', 'http_scans', 'screenshot']}","{'sections': ['general', 'url_list', 'http_scans', 'screenshot'], 'indicator': 'http://104.248.1...",https://otx.alienvault.com/api/v1/indicators/url/http%3A%2F%2F104.248.196.145%2Fapache2/general,0
6,http://ajaraheritage.ge/g7cberv,url,http://ajaraheritage.ge/g7cberv,None,VirusTotal,False,information,No Content,<Response [204]>,https://www.virustotal.com/vtapi/v2/url/report,204
6,http://ajaraheritage.ge/g7cberv,url,http://ajaraheritage.ge/g7cberv,None,XForce,True,information,"{'score': 0, 'cats': None, 'categoryDescriptions': None, 'reason': None, 'reasonDescription': 0,...","{'result': {'url': 'ajaraheritage.ge', 'cats': {}, 'score': None, 'categoryDescriptions': {}}, '...",https://api.xforce.ibmcloud.com/url/http://ajaraheritage.ge/g7cberv,0
8,http://ajaraheritage.ge/g7cberv,url,NaN,None,AzSTI,False,0,Not found.,NaN,ThreatIntelligenceIndicator | where TimeGenerated >= datetime(2021-03-22T18:22:19.058146Z) | w...,0
6,http://ajaraheritage.ge/g7cberv,url,http%3A%2F%2Fajaraheritage.ge%2Fg7cberv,None,OTX,True,high,"{'pulse_count': 2, 'names': ['Locky Ransomware Distribution Sites URL blocklist (LY_DS_URLBL)', ...","{'sections': ['general', 'url_list', 'http_scans', 'screenshot'], 'indicator': 'http://ajaraheri...",https://otx.alienvault.com/api/v1/indicators/url/http%3A%2F%2Fajaraheritage.ge%2Fg7cberv/general,0
4,http://append.pl/srh9xsz,url,http%3A%2F%2Fappend.pl%2Fsrh9xsz,None,OTX,True,warning,"{'pulse_count': 1, 'names': ['Locky Ransomware Distribution Sites URL blocklist (LY_DS_URLBL)'],...","{'sections': ['general', 'url_list', 'http_scans', 'screenshot'], 'indicator': 'http://append.pl...",https://otx.alienvault.com/api/v1/indicators/url/http%3A%2F%2Fappend.pl%2Fsrh9xsz/general,0
4,http://append.pl/srh9xsz,url,http://append.pl/srh9xsz,None,XForce,True,information,"{'score': 0, 'cats': None, 'categoryDescriptions': None, 'reason': None, 'reasonDescription': 0,...","{'result': {'url': 'append.pl', 'cats': {'Software / Hardware': True}, 'score': 1, 'categoryDesc...",https://api.xforce.ibmcloud.com/url/http://append.pl/srh9xsz,0


### Multiple Mixed IoC Types

In [20]:
ioc_mixed = [
    "http://104.248.196.145/apache2",
    "http://ajaraheritage.ge/g7cberv",
    "http://cic-integration.com/hjy93JNBasdas",
    "51.75.29.61",
    "33.44.55.66",
    "52.183.120.194",
    "f8a7135496fd6168df5f0ea21c745db89ecea9accc29c5cf281cdf3145865092",
    "cc2db822f652ca67038ba7cca8a8bde3",
    "ajaraheritage.ge",
]
results = ti_lookup.lookup_iocs(data=ioc_mixed)
results

,Ioc,IocType,SafeIoc,QuerySubtype,Provider,Result,Severity,Details,RawResult,Reference,Status
0,http://104.248.196.145/apache2,url,http%3A%2F%2F104.248.196.145%2Fapache2,None,OTX,True,information,"{'pulse_count': 0, 'sections_available': ['general', 'url_list', 'http_scans', 'screenshot']}","{'sections': ['general', 'url_list', 'http_scans', 'screenshot'], 'indicator': 'http://104.248.1...",https://otx.alienvault.com/api/v1/indicators/url/http%3A%2F%2F104.248.196.145%2Fapache2/general,0
1,http://ajaraheritage.ge/g7cberv,url,http%3A%2F%2Fajaraheritage.ge%2Fg7cberv,None,OTX,True,high,"{'pulse_count': 2, 'names': ['Locky Ransomware Distribution Sites URL blocklist (LY_DS_URLBL)', ...","{'sections': ['general', 'url_list', 'http_scans', 'screenshot'], 'indicator': 'http://ajaraheri...",https://otx.alienvault.com/api/v1/indicators/url/http%3A%2F%2Fajaraheritage.ge%2Fg7cberv/general,0
2,http://cic-integration.com/hjy93JNBasdas,url,http%3A%2F%2Fcic-integration.com%2Fhjy93JNBasdas,None,OTX,True,warning,"{'pulse_count': 1, 'names': ['Locky Ransomware Distribution Sites URL blocklist (LY_DS_URLBL)'],...","{'sections': ['general', 'url_list', 'http_scans', 'screenshot'], 'indicator': 'http://cic-integ...",https://otx.alienvault.com/api/v1/indicators/url/http%3A%2F%2Fcic-integration.com%2Fhjy93JNBasda...,0
3,51.75.29.61,ipv4,51.75.29.61,None,OTX,True,high,"{'pulse_count': 50, 'names': ['IOCs - 202135947 - ANIA Threat Feeds - IP Segment 0', 'IOCs - 202...","{'whois': 'http://whois.domaintools.com/51.75.29.61', 'reputation': 0, 'indicator': '51.75.29.61...",https://otx.alienvault.com/api/v1/indicators/IPv4/51.75.29.61/general,0
4,33.44.55.66,ipv4,33.44.55.66,None,OTX,True,information,"{'pulse_count': 0, 'sections_available': ['general', 'geo', 'reputation', 'url_list', 'passive_d...","{'whois': 'http://whois.domaintools.com/33.44.55.66', 'reputation': 0, 'indicator': '33.44.55.66...",https://otx.alienvault.com/api/v1/indicators/IPv4/33.44.55.66/general,0
5,52.183.120.194,ipv4,52.183.120.194,None,OTX,True,information,"{'pulse_count': 0, 'sections_available': ['general', 'geo', 'reputation', 'url_list', 'passive_d...","{'whois': 'http://whois.domaintools.com/52.183.120.194', 'reputation': 0, 'indicator': '52.183.1...",https://otx.alienvault.com/api/v1/indicators/IPv4/52.183.120.194/general,0
6,f8a7135496fd6168df5f0ea21c745db89ecea9accc29c5cf281cdf3145865092,sha256_hash,f8a7135496fd6168df5f0ea21c745db89ecea9accc29c5cf281cdf3145865092,None,OTX,True,high,"{'pulse_count': 3, 'names': ['Emotet IOCs 2/4/2019', 'Emotet IOCs 2/1/2019', 'Emotet IOCs 1/31/2...","{'sections': ['general', 'analysis'], 'type': 'sha256', 'type_title': 'FileHash-SHA256', 'indica...",https://otx.alienvault.com/api/v1/indicators/file/f8a7135496fd6168df5f0ea21c745db89ecea9accc29c5...,0
7,cc2db822f652ca67038ba7cca8a8bde3,md5_hash,cc2db822f652ca67038ba7cca8a8bde3,None,OTX,True,information,"{'pulse_count': 0, 'sections_available': ['general', 'analysis']}","{'sections': ['general', 'analysis'], 'type': 'md5', 'type_title': 'FileHash-MD5', 'indicator': ...",https://otx.alienvault.com/api/v1/indicators/file/cc2db822f652ca67038ba7cca8a8bde3/general,0
8,ajaraheritage.ge,dns,ajaraheritage.ge,None,OTX,True,high,"{'pulse_count': 27, 'names': ['Veronica.muratori ', 'Veronica.muratori ', 'https://instagram.com...","{'sections': ['general', 'geo', 'url_list', 'passive_dns', 'malware', 'whois', 'http_scans'], 'w...",https://otx.alienvault.com/api/v1/indicators/domain/ajaraheritage.ge/general,0
0,http://104.248.196.145/apache2,url,http://104.248.196.145/apache2,None,VirusTotal,False,information,No Content,<Response [204]>,https://www.virustotal.com/vtapi/v2/url/report,204


## Browsing TI Results
To make it easier to walk through the returned results there is a browser. 
This shows you results aggregated by the IoC value (e.g. an individual IP 
Address or URL) for all providers.

For each provider that returns a result for an IoC, the summarized details 
will be shown in a table below the browse list.
Click on `Raw results from provider...` to see all returned data.

> **Note**: the reference URL may not work if you have not authenticated
> to the service

The value of the selected IoC entry is available as `ti_selector.value`

You can match this back to the original results DataFrame as follows:
```
results[results["Ioc"] == ti_selector.value[0]]
```

In [21]:
from msticpy.nbtools.ti_browser import browse_results
ti_selector = browse_results(data=results, height="200px")
ti_selector

## Specifying Time Ranges
Some providers (currently only AzSTI) support time ranges so that you can specify specific periods to search for.

If a provider does not support time ranges, the parameters will be ignored

In [31]:
from datetime import datetime
search_origin = datetime.now()
q_times = nbwidgets.QueryTime(units="hour", auto_display=True, origin_time=search_origin, max_after=24, max_before=24)

In [32]:
# Using this data range returned no results
ti_lookup.lookup_iocs(data=ioc_ips, providers="AzSTI", start=q_times.start, end=q_times.end).head()

,Ioc,IocType,QuerySubtype,Reference,Result,Status,Severity,Details,RawResult,Provider
0,1.2.3.4,ipv4,None,ThreatIntelligenceIndicator | where TimeGenerated >= datetime(2021-04-21T05:24:53.933864Z) | w...,False,0.0,0.0,Not found.,NaN,AzSTI
1,51.75.29.61,ipv4,None,ThreatIntelligenceIndicator | where TimeGenerated >= datetime(2021-04-21T05:24:53.933864Z) | w...,False,0.0,0.0,Not found.,NaN,AzSTI
2,1.2.3.5,ipv4,None,ThreatIntelligenceIndicator | where TimeGenerated >= datetime(2021-04-21T05:24:53.933864Z) | w...,False,0.0,0.0,Not found.,NaN,AzSTI
3,109.236.89.61,ipv4,None,ThreatIntelligenceIndicator | where TimeGenerated >= datetime(2021-04-21T05:24:53.933864Z) | w...,True,0.0,2.0,"{'Action': 'alert', 'ThreatType': 'Botnet', 'ThreatSeverity': nan, 'Active': True, 'Description'...","{'IndicatorId': '2B3D1B719533D10EF42E0BA1948BDE099A83C38BAF53E8369CB9651A890A0CD2', 'TimeGenerat...",AzSTI
4,109.236.89.61,ipv4,None,ThreatIntelligenceIndicator | where TimeGenerated >= datetime(2021-04-21T05:24:53.933864Z) | w...,True,0.0,2.0,"{'Action': 'alert', 'ThreatType': 'Botnet', 'ThreatSeverity': nan, 'Active': True, 'Description'...","{'IndicatorId': '30196A288FA811E8ACCCC84C434E3AD15F175B17BE2D4458DD3BC660073C1365', 'TimeGenerat...",AzSTI


In [34]:
from datetime import datetime
search_origin = datetime.now()
q_times = nbwidgets.QueryTime(units="day", auto_display=True, origin_time=search_origin, max_after=24, max_before=24)

In [36]:
# Using a wider ranges produces results
ti_lookup.lookup_iocs(data=ioc_ips, providers="AzSTI", start=q_times.start, end=q_times.end)

,Ioc,IocType,QuerySubtype,Reference,Result,Status,Severity,Details,RawResult,Provider
0,1.2.3.4,ipv4,None,ThreatIntelligenceIndicator | where TimeGenerated >= datetime(2021-04-12T11:25:07.689524Z) | w...,False,0.0,0.0,Not found.,NaN,AzSTI
1,51.75.29.61,ipv4,None,ThreatIntelligenceIndicator | where TimeGenerated >= datetime(2021-04-12T11:25:07.689524Z) | w...,False,0.0,0.0,Not found.,NaN,AzSTI
2,1.2.3.5,ipv4,None,ThreatIntelligenceIndicator | where TimeGenerated >= datetime(2021-04-12T11:25:07.689524Z) | w...,False,0.0,0.0,Not found.,NaN,AzSTI
3,109.236.89.61,ipv4,None,ThreatIntelligenceIndicator | where TimeGenerated >= datetime(2021-04-12T11:25:07.689524Z) | w...,True,0.0,2.0,"{'Action': 'alert', 'ThreatType': 'Botnet', 'ThreatSeverity': nan, 'Active': True, 'Description'...","{'IndicatorId': '2E9C9B5884CE9D1AAECA21C5EB772C30C3C84093C452AA038868AD4EEF50E83E', 'TimeGenerat...",AzSTI
4,109.236.89.61,ipv4,None,ThreatIntelligenceIndicator | where TimeGenerated >= datetime(2021-04-12T11:25:07.689524Z) | w...,True,0.0,2.0,"{'Action': 'alert', 'ThreatType': 'Botnet', 'ThreatSeverity': nan, 'Active': True, 'Description'...","{'IndicatorId': 'AA38B8D61F8587B779AABF6C4F2DBD226C7704600A5CBBB7779A7D0E7AFA05C4', 'TimeGenerat...",AzSTI
5,109.236.89.61,ipv4,None,ThreatIntelligenceIndicator | where TimeGenerated >= datetime(2021-04-12T11:25:07.689524Z) | w...,True,0.0,2.0,"{'Action': 'alert', 'ThreatType': 'Botnet', 'ThreatSeverity': nan, 'Active': True, 'Description'...","{'IndicatorId': 'E9DD356FF6855D16952067E4A35F0AAA1A8FF4E955E20B249131BD0EF7115A1E', 'TimeGenerat...",AzSTI
6,109.236.89.61,ipv4,None,ThreatIntelligenceIndicator | where TimeGenerated >= datetime(2021-04-12T11:25:07.689524Z) | w...,True,0.0,2.0,"{'Action': 'alert', 'ThreatType': 'Botnet', 'ThreatSeverity': nan, 'Active': True, 'Description'...","{'IndicatorId': '0612D0A787F9B2EA5FC1206C3727B3ADAD45D8AEDE2D90ADEC1C3B1590A609E0', 'TimeGenerat...",AzSTI
7,109.236.89.61,ipv4,None,ThreatIntelligenceIndicator | where TimeGenerated >= datetime(2021-04-12T11:25:07.689524Z) | w...,True,0.0,2.0,"{'Action': 'alert', 'ThreatType': 'Botnet', 'ThreatSeverity': nan, 'Active': True, 'Description'...","{'IndicatorId': '4DDD4BC9AD3844756B4DF7FAB23E9E9B917124D0A257BB405EF5715DEAA3D61B', 'TimeGenerat...",AzSTI
8,109.236.89.61,ipv4,None,ThreatIntelligenceIndicator | where TimeGenerated >= datetime(2021-04-12T11:25:07.689524Z) | w...,True,0.0,2.0,"{'Action': 'alert', 'ThreatType': 'Botnet', 'ThreatSeverity': nan, 'Active': True, 'Description'...","{'IndicatorId': 'CAFF80F6C8FD64CDA73AE68E382C6A7B6C94CA257C897324E6DB78D8C2106588', 'TimeGenerat...",AzSTI
9,109.236.89.61,ipv4,None,ThreatIntelligenceIndicator | where TimeGenerated >= datetime(2021-04-12T11:25:07.689524Z) | w...,True,0.0,2.0,"{'Action': 'alert', 'ThreatType': 'Botnet', 'ThreatSeverity': nan, 'Active': True, 'Description'...","{'IndicatorId': 'C72FAFC4A20054F73B343418F5EBE887610D257FFD51439392CC0376AB3B7173', 'TimeGenerat...",AzSTI
